#### Declare necessary modules/links/directories

In [60]:
from bs4 import BeautifulSoup
import requests
import re
import json
import scrapping_functions as sf
reload(sf)
from selenium import webdriver
#Start = signifies the listing to start it, increases in increments of 10 per page
#End = 990
target_url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Washington+DC&start='
base = 'https://www.yelp.com/'

In [2]:
#Pull in a list of links from the target url
link_list = []
for x in range(507, 991):
    target = target_url + str(x)
    raw_html = requests.get(target)
    soup = BeautifulSoup(raw_html.text, 'html.parser')
    link_list = sf.biz_links(soup, link_list)


In [3]:
#Remove extraneous links, keep only business links
for link in link_list:
    if re.search("adredir", link):
        link_list.pop(link_list.index(link))

In [4]:
#Write all links to a text file
biz_links = open('cleanbiz_links_2.txt', 'w')
for item in link_list:
    biz_links.write("%s\n" % item)
biz_links.close()

We would like to construct the following dictionary:

biz_dict = {biz_name: {"city": "Washington", "state": "DC", "category_aliases": "newamerican,breakfast_brunch", "biz_id": "wO-7cBBOYUdiLflpuRsu9A", "latitude": 38.90842, "biz_name": "The Bird", "city_state": "Washington, DC", "longitude": -77.026685, "geoquad": 12845454}}

unique_id = 5D32F13B349CE2AD

#### Algorithm design:

For each link in biz_links:
    > set biz_name = replace("_", '-' in link)
    > find the unique_id
    > assign biz_dict[biz_name] = dictionary(biz_name)

In [87]:
link_file = open("cleanbiz_links.txt", "r")
link_list = link_file.read().split('\n')
link_list = list(set(link_list))

In [11]:
# raw_html = requests.get(base + link_list[0])
# soup = BeautifulSoup(raw_html.text, 'html.parser')
# soup

In [89]:
biz_dict = {}

for link in link_list:
    biz_name = link.replace('?osq=Restaurants', '')
    biz_name = re.sub("-washington*", '', biz_name)
    biz_dict[biz_name] = {}
    
    raw_html = requests.get(base + link)
    soup = BeautifulSoup(raw_html.text, 'html.parser')
    parse_string = str(soup.find_all("script"))
    parse_list = parse_string.split('<script>')
    
    try:
        match = re.sub(".*,null,",'', [s for s in parse_list if re.search(".*latitude.*", s)][0])
    except IndexError:
        "The regexp is matching too many JSON entries, please take a look at " + str(link)
    
    biz_dict[biz_name] = json.loads(re.sub(", \"geoquad\".*",'', match) + '}')

In [90]:
#Output JSON file of all the business details
with open('biz_details_2.json', 'w') as outfile:
    json.dump(biz_dict, outfile)

In [91]:
biz_json = json.load(open("/Users/robertsonwang/Desktop/Python/Yelp/Yelp_scrapper/biz_details.json"))
agg_dict = dict(biz_json, **biz_dict)
with open('biz_details_agg.json', 'w') as outfile:
    json.dump(agg_dict, outfile)
    
    